In [7]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml import command
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
credential = DefaultAzureCredential()
workspace_ml_client = None
try:
    workspace_ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)

Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [25]:
subscription_id = workspace_ml_client.subscription_id
resource_group = workspace_ml_client.resource_group_name
workspace_name = workspace_ml_client.workspace_name

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
endpoint_name = "deepseek-quen-15B-endpoint"

In [6]:
model_path = "model"
import os
os.makedirs(base_dir, exist_ok=True)

In [4]:
from huggingface_hub import snapshot_download
snapshot_download(
            repo_id=model_name,
            local_dir=model_path,
            #ignore_patterns=["*.safetensors", "*.bin"]
        )

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 9 files: 100%|██████████| 9/9 [01:38<00:00, 10.95s/it]


'/mnt/batch/tasks/shared/LS_root/mounts/clusters/mrajguru3/code/Users/mrajguru/deepseek/model'

In [9]:
model_path

'model'

In [11]:
from azure.ai.ml.entities import Model, Environment
from azure.ai.ml.constants import AssetTypes
import os
from uuid import uuid4
local_model = Model(
    path=model_path,
    type=AssetTypes.CUSTOM_MODEL,
    name="r1-distil-quen-15b",
    version="1",
    description="Model created from local file for text generation deployment.",
)
model = workspace_ml_client.models.create_or_update(local_model)

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mrajguru3/code/Users/mrajguru/deepseek/model' 'https://amldevsweden3845944479.blob.core.windows.net/azureml-blobstore-06e6f83f-87ce-4aa8-8df6-8e5cda0ac300/LocalUpload/00cca47d13f6a3b1f8b424964570907a/model' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.


In [12]:
from azure.ai.ml.entities import Environment, BuildContext
env_docker_image = Environment(
            build=BuildContext(path="environment"),
            name="vllm-custom",
            description="Environment created from a Docker context.",
            inference_config={
                "liveness_route": {
                    "port": 8000,
                    "path": "/health",
                },
                "readiness_route": {
                    "port": 8000,
                    "path": "/health",
                },
                "scoring_route": {
                    "port": 8000,
                    "path": "/",
                },
            },
        )
env_asset = workspace_ml_client.environments.create_or_update(env_docker_image)

Uploading environment (0.0 MBs): 100%|██████████| 612/612 [00:00<00:00, 3368.01it/s]




In [26]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    IdentityConfiguration,
    ManagedIdentityConfiguration,
)

# Check if the endpoint already exists in the workspace
try:
    endpoint = workspace_ml_client.online_endpoints.get(endpoint_name)
except:
    endpoint = ManagedOnlineEndpoint(
        name=endpoint_name,
        description=f"Test endpoint for {model.name}",
    )

In [27]:
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [58]:
env_vars = {
    "MODEL_NAME": "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
    "VLLM_ARGS": "--max-model-len 16000 --enforce-eager",
}
deployment_env_vars = {**env_vars}

In [72]:
import time
from azure.ai.ml.entities import (    
    OnlineRequestSettings,
    CodeConfiguration,
    ManagedOnlineDeployment,
    ProbeSettings,
    Environment
)

t0 = time.time()
deployment = ManagedOnlineDeployment(
    name=  "deepseek-quen-15b-deployment",
    endpoint_name=endpoint_name,
    #model=model,
    instance_type="Standard_NC24ads_A100_v4",
    instance_count=1,
    #code_configuration=code_configuration,
    environment_variables=deployment_env_vars,    
    environment=env_asset,
    # scoring_script="score.py",
    # code_path="./src_serve",
    request_settings=OnlineRequestSettings(
        max_concurrent_requests_per_instance=2,
        request_timeout_ms=50000, 
        max_queue_wait_ms=60000
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=5,
        success_threshold=1,
        timeout=10,
        period=30,
        initial_delay=120
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=120,
    ),
)

# Trigger the deployment creation
try:
    workspace_ml_client.begin_create_or_update(deployment).wait()
except Exception as err:
    raise RuntimeError(
        f"Deployment creation failed. Detailed Response:\n{err}"
    ) from err
    
endpoint.traffic = {"deepseek-quen-15b-deployment": 100}
endpoint_poller = workspace_ml_client.online_endpoints.begin_create_or_update(endpoint)

Check: endpoint deepseek-quen-15b-endpoint exists


.............

In [73]:
endpoint_results = endpoint_poller.result()
endpoint_name = endpoint_results.name
keys = workspace_ml_client.online_endpoints.get_keys(name=endpoint_name)
primary_key = keys.primary_key

In [74]:
from openai import OpenAI

url = os.path.join(endpoint_results.scoring_uri, "v1")
endpoint_name = (
    endpoint_results.name if endpoint_name is None else endpoint_name
)
keys = workspace_ml_client.online_endpoints.get_keys(name=endpoint_name)
vllm_client = OpenAI(base_url=url, api_key=primary_key)
model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"

In [82]:
# Create your prompt
system_message = """You are a helpful Assistant"""
user_message = f"""write a python code to read a csv"""
response = vllm_client .chat.completions.create(
    model=model_path,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.7,
    max_tokens=4000,
    stream=True,  # Stream the response
)
 
print("Streaming response:")
for chunk in response:
    delta = chunk.choices[0].delta
    if hasattr(delta, "content"):
        print(delta.content, end="", flush=True)

Streaming response:
<think>
Okay, the user wants me to write a Python code to read a CSV file. I remember that Python has a built-in module called csv which is good for handling CSV files. But wait, I've also heard that pandas is a library that makes working with data easier. Maybe I should provide both methods so the user can choose depending on their needs.

First, I'll start with the standard library approach using csv. I'll need to import the csv module. Then, I'll open the file using a with statement to ensure it's properly closed after reading. I'll use csv.reader to read the file, and probably read the header first so the user can see the columns. Then, I'll loop through each row and print it out.

Next, for the pandas method, I'll need to import pandas as pd. Then, I can read the CSV file using pd.read_csv and store it in a DataFrame. I should print out the first few rows using head() so the user can see the data quickly. Also, maybe include some comments about installing panda